# Download the datasets

### 21 Mil WikiDPR

In [ ]:
import pandas
from datasets import load_dataset

dataset = load_dataset("wiki_dpr", "psgs_w100.multiset.no_index", streaming= True)

In [ ]:
import numpy as np
print(next(iter(dataset['train'])))
# embeds = np.array([doc['emb'] for doc in dataset['train']])
# print("Embeddings:")
# print(embeds)


### Cohere wiki 485k dataset

In [1]:
import pandas
from datasets import load_dataset

docs = load_dataset(f"Cohere/wikipedia-22-12-simple-embeddings", split="train")

Old caching folder C:\Users\BoodyBeeh\.cache\huggingface\datasets\Cohere___parquet\Cohere--wikipedia-22-12-simple-embeddings-94deea3d55a22093\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec for dataset parquet exists but no data were found. Removing it. 


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/485859 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/BoodyBeeh/.cache/huggingface/datasets/Cohere___parquet/Cohere--wikipedia-22-12-simple-embeddings-94deea3d55a22093/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


c:\Users\BoodyBeeh\anaconda3\Lib\site-packages\datasets\table.py:1407: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
for doc in docs:

    with open("embeds2.txt", "a") as f:
        f.write(str(doc['emb']))
        f.write('\n')

# Attempts


### Convert file to binary


In [ ]:
# pickle dump the docs
import pickle
with open('wikiCohere.bin', 'wb') as f:
    pickle.dump(docs, f)

In [5]:
import numpy as np
embeds = np.array([doc['emb'] for doc in docs])
for emb in embeds:
    with open('try.bin', 'a') as f:
        f.write(str(emb))

# KD tree

### Constructing KD tree

In [ ]:
from sklearn.neighbors import KDTree
from scipy.spatial import cKDTree
tree = KDTree(embeds) # leafSize = 16
# tree = cKDTree(embeds)

In [ ]:
tree_data, index, tree_nodes, node_bounds = tree.get_arrays()
tree_nodes

# K means

### Running K means on each batch
- 100k batches
- each batch 100 clusters

In [47]:
# loop on folders
import os
import numpy as np
from sklearn.cluster import KMeans

np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # ensure numpy array won't truncate

with open("./batches/1-embeds-batch.txt", "r") as f:
    # read each line and convert to numpy array
    lines = [np.fromstring(line, sep=" ") for line in f]
    # convert to numpy array
    embeds = np.array(lines)
    # apply kmeans on embeds and get the centroids
    kmeans = KMeans(n_clusters = 100).fit(embeds)
    with open("./batches/centroids1.txt", "a") as f:
        f.write(str(kmeans.cluster_centers_))


c:\Users\BoodyBeeh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Collecting all centroid files into one array
* Make sure that centroids files don't have any squared brackets

In [53]:
centroids = np.loadtxt("./batches/centroids0.txt")
centroids = np.concatenate((centroids, np.loadtxt("./batches/centroids1.txt")), axis=0)
centroids = np.concatenate((centroids, np.loadtxt("./batches/centroids2.txt")), axis=0)
centroids = np.concatenate((centroids, np.loadtxt("./batches/centroids3.txt")), axis=0)
centroids = np.concatenate((centroids, np.loadtxt("./batches/centroids4.txt")), axis=0)

Applying K-means on all the centroids

In [55]:
# apply kmeans on embeds and get the centroids
kmeans = KMeans(n_clusters = 100).fit(centroids)
with open("./batches/final-centroids.txt", "a") as f:
    f.write(str(kmeans.cluster_centers_))

c:\Users\BoodyBeeh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\BoodyBeeh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Assigning labels to each batch according to the final centroids

In [73]:
def assign_labels(X, centroids):
    labels = np.empty(X.shape[0], dtype=np.int64)
    for i, x in enumerate(X):
        distances = np.linalg.norm(x - centroids, axis=1)
        labels[i] = np.argmin(distances)
    return labels


with open("./batches/4-embeds-batch.txt", "r") as f:
    # read each line and convert to numpy array
    lines = [np.fromstring(line, sep=" ") for line in f]
    # convert to numpy array
    embeds = np.array(lines)
    # print(embeds.shape)
    centroids = np.loadtxt("./batches/final-centroids.txt")
    labels = assign_labels(embeds, centroids)
    with open("./batches/labels4.txt", "a") as f:
        f.write(str(labels))

Constructing clusters

In [87]:
# opening the embeds file
with open("./batches/dataset/1-embeds-batch.txt", "r") as f:
    lines = [np.fromstring(line, sep=" ") for line in f]
    embeds = np.array(lines)

# read the labels and convert it to a list
with open("./batches/labels/labels1.txt", "r") as f:
    labels =[int(x) for x in next(f).split()]   

# loop on the labels 
for i in range(100):
    indices = [j for j, x in enumerate(labels) if x == i]
    current_embeds = embeds[indices]
    with open(f"./batches/clusters/cluster{i}.txt", "a") as f:
        f.write(str(current_embeds))

In [ ]:
for i in range(100):
    with open(f"./batches/clusters/cluster{i}.txt", "a") as f:
        f.write("\n")